<a href="https://colab.research.google.com/github/eugeneegbe/Greedy_rule_classifier/blob/main/GreedyRuleListClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip install imodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 10.3 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from imodels import GreedyRuleListClassifier
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
df = pd.read_csv('/content/drive/MyDrive/Paderborn University/XAI/Exercise-2/bike.csv')
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,temp,atemp,hum,windspeed,casual,registered,cnt,workday,weather,days_since_2011,cnt_2d_bfr
0,3,2011-01-03,WINTER,2011,JAN,N,MON,1.229108,22.439770,43.7273,16.636703,120,1229,1349,Y,GOOD,2,985
1,4,2011-01-04,WINTER,2011,JAN,N,TUE,1.400000,23.212148,59.0435,10.739832,108,1454,1562,Y,GOOD,3,801
2,5,2011-01-05,WINTER,2011,JAN,N,WED,2.666979,23.795180,43.6957,12.522300,82,1518,1600,Y,GOOD,4,1349
3,6,2011-01-06,WINTER,2011,JAN,N,THU,1.604356,23.929106,51.8261,6.000868,88,1518,1606,Y,GOOD,5,1562
4,7,2011-01-07,WINTER,2011,JAN,N,FRI,1.236534,23.100526,49.8696,11.304642,148,1362,1510,Y,MISTY,6,1600


# Discretize The Target
Assign to **1** if its greater than mean and **0** otherwise.

In [14]:

target = df['cnt']
cnt_median = np.median(target)

# Discretize based on the median values for cnt
df['cnt'] = df['cnt'].apply(lambda x: 1 if x > cnt_median else 0)

# Apply quartile cut to continues features (temp, hum, windspeed, cnt_2_before)

This is essential for later use to apply **sort of coding** for our rules decision for these features

In [15]:
# Quartile cut for continuous features for different categories low, mediaum, high
df['temp'] = pd.qcut(df['temp'], q=3, labels=['TEM_LOW', 'TEMP_MEDIUM', 'TEMP_HIGH'])
df['hum'] = pd.qcut(df['hum'], q=3, labels=['HUM_LOW', 'HUM_MEDIUM', 'HUM_HIGH'])
df['windspeed'] = pd.qcut(df['windspeed'], q=3, labels=['WIND_SPEED_LOW', 'WIND_SPEED_MEDIUM', 'WIND_SPEED_HIGH'])
df['cnt_2d_bfr'] = pd.qcut(df['cnt_2d_bfr'], q=3, labels=['CNT_2_BEFORE_LOW', 'CNT_2_BEFORE_MEDIUM', 'CNT_2_BEFORE_HIGH'])

df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,temp,atemp,hum,windspeed,casual,registered,cnt,workday,weather,days_since_2011,cnt_2d_bfr
0,3,2011-01-03,WINTER,2011,JAN,N,MON,TEM_LOW,22.439770,HUM_LOW,WIND_SPEED_HIGH,120,1229,0,Y,GOOD,2,CNT_2_BEFORE_LOW
1,4,2011-01-04,WINTER,2011,JAN,N,TUE,TEM_LOW,23.212148,HUM_MEDIUM,WIND_SPEED_MEDIUM,108,1454,0,Y,GOOD,3,CNT_2_BEFORE_LOW
2,5,2011-01-05,WINTER,2011,JAN,N,WED,TEM_LOW,23.795180,HUM_LOW,WIND_SPEED_MEDIUM,82,1518,0,Y,GOOD,4,CNT_2_BEFORE_LOW
3,6,2011-01-06,WINTER,2011,JAN,N,THU,TEM_LOW,23.929106,HUM_LOW,WIND_SPEED_LOW,88,1518,0,Y,GOOD,5,CNT_2_BEFORE_LOW
4,7,2011-01-07,WINTER,2011,JAN,N,FRI,TEM_LOW,23.100526,HUM_LOW,WIND_SPEED_MEDIUM,148,1362,0,Y,MISTY,6,CNT_2_BEFORE_LOW


# Mapping dicrete feature Categories
These categories don't have a wider range


In [16]:
# Map month values to numeric values
month_map = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4,
    'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8,
    'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}
df['mnth'] = df['mnth'].map(month_map)


df['seasonSUMMER'] = np.where(df['season'] == 'SUMMER', 1, 0)
df['seasonWINTER'] = np.where(df['season'] == 'WINTER', 1, 0)
df['seasonSPRING'] = np.where(df['season'] == 'SPRING', 1, 0)
df['seasonFALL'] = np.where(df['season'] == 'FALL', 1, 0)


# Holiday
df['holiday'] = np.where(df['holiday'] == 'Y', 1, 0)

# create a weekday map

weekday_map = {
    'MON': 1, 'TUE': 2, 'WED': 3, 'THU': 4,
    'FRI': 5, 'SAT': 6, 'SUN': 7
}
df['weekday'] = df['weekday'].map(weekday_map)
df['time_of_week'] = pd.qcut(df['weekday'], q=3, labels=['START_WEEK', 'MID_WEEK', 'WEEK_END'])

# Workday
df['workday'] = np.where(df['workday'] == 'Y', 1, 0)


df = df.drop(columns=['dteday', 'yr', 'season', 'instant', 'atemp', 'casual', 'registered', 'dteday', 'days_since_2011'], axis=1)

df.head()

,mnth,holiday,weekday,temp,hum,windspeed,cnt,workday,weather,cnt_2d_bfr,seasonSUMMER,seasonWINTER,seasonSPRING,seasonFALL,time_of_week
0,1,0,1,TEM_LOW,HUM_LOW,WIND_SPEED_HIGH,0,1,GOOD,CNT_2_BEFORE_LOW,0,1,0,0,START_WEEK
1,1,0,2,TEM_LOW,HUM_MEDIUM,WIND_SPEED_MEDIUM,0,1,GOOD,CNT_2_BEFORE_LOW,0,1,0,0,START_WEEK
2,1,0,3,TEM_LOW,HUM_LOW,WIND_SPEED_MEDIUM,0,1,GOOD,CNT_2_BEFORE_LOW,0,1,0,0,START_WEEK
3,1,0,4,TEM_LOW,HUM_LOW,WIND_SPEED_LOW,0,1,GOOD,CNT_2_BEFORE_LOW,0,1,0,0,MID_WEEK
4,1,0,5,TEM_LOW,HUM_LOW,WIND_SPEED_MEDIUM,0,1,MISTY,CNT_2_BEFORE_LOW,0,1,0,0,MID_WEEK


# Encode categorical features (Effect encoding)

Choice of Effect encoding was chosen because it gave ME better results in the previous lab

In [17]:


weather_dummies = pd.get_dummies(df['weather'], drop_first=True).astype(int)
temp_dummies = pd.get_dummies(df['temp'], drop_first=True).astype(int)
hum_dummies = pd.get_dummies(df['hum'], drop_first=True).astype(int)
windspeed_dummies = pd.get_dummies(df['windspeed'], drop_first=True).astype(int)
time_of_week_dummies = pd.get_dummies(df['time_of_week'], drop_first=True).astype(int)

cnt_2d_bfr_dummies = pd.get_dummies(df['cnt_2d_bfr'], drop_first=True).astype(int)


df = df.drop(columns=['weekday', 'weather', 'temp', 'hum', 'windspeed', 'cnt_2d_bfr', 'time_of_week'], axis=1)
df = pd.concat([df, weather_dummies, temp_dummies, hum_dummies, windspeed_dummies, cnt_2d_bfr_dummies, time_of_week_dummies], axis=1)

target = df.pop('cnt')
df.head()

,mnth,holiday,workday,seasonSUMMER,seasonWINTER,seasonSPRING,seasonFALL,GOOD,MISTY,TEMP_MEDIUM,TEMP_HIGH,HUM_MEDIUM,HUM_HIGH,WIND_SPEED_MEDIUM,WIND_SPEED_HIGH,CNT_2_BEFORE_MEDIUM,CNT_2_BEFORE_HIGH,MID_WEEK,WEEK_END
0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,1,0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0
2,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0
4,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0


# Apply the Greedy Rule List Classifier


In [19]:

grlc = GreedyRuleListClassifier(max_depth=5)

predictions = grlc.fit(df, target)

print(grlc)

> ------------------------------
> Greedy Rule List
> ------------------------------
↓
29.9% risk (728 pts)
	if CNT_2_BEFORE_HIGH ==> 90.10000000000001% risk (243 pts)
↓
9.47% risk (485 pts)
	if CNT_2_BEFORE_MEDIUM ==> 50.4% risk (242 pts)
↓
6.55% risk (243 pts)
	if TEMP_HIGH ==> 57.099999999999994% risk (14 pts)
↓
2.41% risk (229 pts)
	if TEMP_MEDIUM ==> 17.5% risk (63 pts)
↓
0.0% risk (166 pts)
	if seasonFALL ==> 12.9% risk (31 pts)
↓
0.0% risk (135 pts)



# Intepretation
1. **What does risk mean?**

Risk here tells us the number of points caught by a certain rule which contribute to the positive side of our classification.

2. **What does pts mean**

``pts`` refers to the data points that satisfy the rule's condition (this could be seen as coverage)

Example: CNT_2_BEFORE_HIGH ==> 90.10000000000001% risk (243 pts) , means
- 243 pts matched this rule,
- 90% of 243 (i.e 218) of 243 positively contribute to the prediction
- **Accuracy** == **risk** (since we are predicting positives).

3. **How are “risk” and “pts” related to the “coverage” and “accuracy” of a rule as introduced in the lecture?**

* pts == coverage (number of data points which match a rule)
* risk == accuracy

Note: if we were predicting the reverse, that is decrease in bike count, the accuracy would be 100% - 90% for the example rule (this gives the probability that we predict a drop in bike count.